In [ ]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")
os.environ["OPENAI_API_KEY"] = "sk-jxKghxq3MMF3aC1J5c75666dD3D04e2dBaB36eE75b72Dd89"
os.environ["OPENAI_API_BASE"] = "https://ai-yyds.com/v1"

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
print(api_key)

In [ ]:
import os

# 设置HTTP代理
os.environ['HTTP_PROXY'] = "127.0.0.1:7890"
# 设置HTTPS代理
os.environ['HTTPS_PROXY'] = "127.0.0.1:7890"

# 验证设置
print("HTTP_PROXY:", os.environ.get('HTTP_PROXY'))
print("HTTPS_PROXY:", os.environ.get('HTTPS_PROXY'))


# LCEL(LangChain Expression Language)
- 一个最简单示例
<hr>

In [ ]:
#定义llm
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于 {topic}的笑话")
model = ChatOpenAI(model="gpt-4", max_retries=3, temperature=0, request_timeout=600)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "冰激凌"})

Prompt

In [ ]:
prompt_value = prompt.invoke({"topic": "猫"})
prompt_value

In [ ]:
prompt_value.to_messages()

In [ ]:
prompt_value.to_string()

model

In [ ]:
message = model.invoke(prompt_value)
message

使用llm的区别

In [ ]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

Output parser

In [ ]:
output_parser.invoke(message)

LCEL的Pipeline：
- ![Alt Text](lcel01.png)

# RAG Search Example
- 建立向量数据
- 使用RAG增强
<hr>

In [ ]:
from operator import itemgetter

from IPython import embed
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("where did harrison work?")

自定义也非常简单

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language")
    }
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
chain.invoke({"question": "where did harrison work", "language": "chinese"})

# LCEL接口
- 输入格式
- 输出格式
- 8种不同的接口方式
<hr>

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

prompt = ChatPromptTemplate.from_template("给我讲一个关于 {topic}的笑话")
chain = prompt | model

input schema

In [ ]:
chain.input_schema.schema()

In [ ]:
prompt.input_schema.schema()

In [ ]:
model.input_schema.schema()

Output Schema

In [ ]:
chain.output_schema.schema()

Stream（流式）

In [ ]:
for s in chain.stream({"topic" : "熊"}):
    print(s.content, end="", flush=True)

Invoke

In [ ]:
chain.invoke({"topic" : "熊"})

Batch

In [ ]:
chain.batch([{"topic": "熊"}, {"topic": "猫"}])

In [ ]:
#max_concurrency控制并发数
chain.batch([{"topic": "熊"}, {"topic": "猫"}, {"topic": "狗"}], config={"max_concurrency": 2})

Async Stream 异步

In [ ]:
async for s in chain.astream({"topic": "女人"}):
    print(s.content, end="", flush=True)   

Async Invoke

In [ ]:
await chain.ainvoke({"topic": "男人"})

Async Batch

In [ ]:
await chain.abtch([{"topic": "熊"},{"topic": "女人"}])

异步获取中间步骤

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """基于下面的上下文来回答问题:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["柯基犬是一种中型家庭宠物犬"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question" : RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
async for chunk in retrieval_chain.astream_log("柯基是什么？", include_names=["Docs"]):
    print("-" * 40)
    print(chunk)

只看状态值

In [ ]:

async for chunk in retrieval_chain.astream_log(
    "柯基是什么?", include_names=["Docs"], diff=False
):
    print("-" * 70)
    print(chunk)

并行支持

In [ ]:
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("给我讲一个关于 {topic}的笑话") | model
chain2 = (
    ChatPromptTemplate.from_template("写两行关于{topic}的诗歌")
    | model
)
combined = RunnableParallel([chain1, chain2])

In [ ]:
%%time
chain1.invoke({"topic": "熊"})

In [ ]:
%%time
chain2.invoke({"topic": "熊"})

并行执行

In [ ]:
%%time
combined.invoke({"topic": "熊"})

并行批处理，适用于大量生成

In [ ]:
%%time
chain1.batch([{"topic": "熊"}, {"topic": "猫"}])

In [ ]:
%%time
chain2.batch([{"topic": "熊"}, {"topic": "猫"}])

并行执行

%%time
combined.batch([{"topic": "熊"}, {"topic": "猫"}])

# Prompt+LLM
基本构成： 
PromptTemplate / ChatPromptTemplate -> LLM / ChatModel -> OutputParser

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于{foo}的笑话")
model = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
chain = prompt | model

In [ ]:
chain.invoke({"foo": "狗熊"})

自定义停止输出符

In [ ]:
chain = prompt | model.bind(stop=["\n"])

In [ ]:
chain.invoke({"foo": "狗熊"})

兼容openai函数调用的方式

In [ ]:
functions = [
    {
        "name": "joke",
        "description": "讲笑话",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "笑话的开头"},
                "punchline": {
                    "type": "string",
                    "description": "爆梗的结尾",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [ ]:
chain.invoke({"foo": "男人"}, config={})

输出解析器


In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

In [ ]:
chain.invoke({"foo": "女人"})

与函数调用混合使用

In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [ ]:
chain.invoke({"foo": "女人"})

In [ ]:
#只输出setup
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain = (
    {"foo": RunnablePassthrough()} #使用RunnablePassthrough()跳过prompt
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="punchline") # 定义输出的key
)

In [ ]:
#chain.invoke({"foo": "女人"})
#使用RunnablePassthrough()跳过prompt
chain.invoke("女人")

# 使用Runnables来连接多链结构

In [ ]:
from operator import itemgetter #获取可迭代对象中指定索引或键对应的元素

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("{person}来自于哪个城市?")
prompt2 = ChatPromptTemplate.from_template("{city}属于哪个省? 用{language}来回答")

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}#获取invoke中的language
    | prompt2
    | model
    | StrOutputParser()
)
chain1.invoke({"person": "马化腾"})
chain2.invoke({"person": "马化腾", "language": "中文"})

In [ ]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "生成一个{attribute}属性的颜色。除了返回这个颜色的名字不要做其他事:"
)
prompt2 = ChatPromptTemplate.from_template(
    "什么水果是这个颜色:{color},只返回这个水果的名字不要做其他事情:"
)
prompt3 = ChatPromptTemplate.from_template(
    "哪个国家的国旗有这个颜色:{color},只返回这个国家的名字不要做其他事情:"
)
prompt4 = ChatPromptTemplate.from_template(
    "有这个颜色的水果是{fruit},有这个颜色的国旗是{country}？"
)

model_parser = model | StrOutputParser()
# 生成一个颜色
color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [ ]:
question_generator.invoke("强烈的")

多链执行与结果合并

      输入
      / \
     /   \
 分支1   分支2
     \   /
      \ /
    合并结果

唯物辩证链

In [ ]:
planner = (
    ChatPromptTemplate.from_template("生成一个关于{input}的论点")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "列出以下内容的优点或积极方面:{base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出以下内容的缺点或消极方面:{base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "积极:\n{results_1}\n\n消极:\n{results_2}"),
            ("system", "根据评论生成最终的回复"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [ ]:
chain.invoke({"input": "生孩子"})

## 查询SQL

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [ ]:
db.get_table_info()

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4",
    temperature=0,
)

def get_schema(_):
    return db.get_table_info()



sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [ ]:
sql_response.invoke({"question": "How many artists are there?"})

In [ ]:
def run_query(query):
    print(query)
    return db.run(query)

template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
)

In [ ]:
full_chain.invoke({"question": "How many artists are there?"})

## 自定义输出解析器
- python编程助手

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

template = """根据用户需求帮助用户编写python代码. 

只需要返回makedown格式的python代码, 比如:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOpenAI(
    model="gpt-4",
    temperature=0,
)

#自定义输出解析，只返回python代码
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

#定义链
chain = prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run

In [ ]:
chain.invoke({"input": "从10里面循环找出所有质数"})

# LCEL:memory添加方式
<hr>

In [ ]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个乐于助人的机器人"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [ ]:
memory = ConversationBufferMemory(return_messages=True)

In [ ]:
memory.load_memory_variables({})

增加一条链

In [ ]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [ ]:
inputs = {"input": "你好我是tomie"}
response = chain.invoke(inputs)
response

In [ ]:
#保存记忆
memory.save_context(inputs, {"output": response.content})
memory.load_memory_variables({})

In [ ]:
inputs = {"input": "我叫什么名字?"}
response = chain.invoke(inputs)
response

使用Redis来实现长时记忆

In [ ]:
from typing import Optional

from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.chat_models import ChatOpenAI
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个擅长{ability}的助手"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI(model="gpt-4-1106-preview",temperature=0)

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    #使用redis存储聊天记录
    lambda session_id: RedisChatMessageHistory(session_id, url="redis://localhost:6379/0"),
    input_messages_key="question",
    history_messages_key="history",
)

In [ ]:
#每次调用都会保存聊天记录，需要有对应的session_id
chain_with_history.invoke(
    {"ability": "历史", "question": "中国建都时间最长的城市是哪个?"},
    config={"configurable": {"session_id": "tomiezhang"}},
)

In [ ]:
chain_with_history.invoke(
    {"ability": "历史", "question": "它有多少年建都历史？"},
    config={"configurable": {"session_id": "tomiezhang"}},
)

# 第一个Agent
- 注意当前使用的langchain版本为0.1.4，调用方式和之前旧版本有所差异

In [ ]:

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import load_tools
from langchain.agents import create_openai_functions_agent #不同的agent有不同的创建方式
from langchain.agents import AgentExecutor
#创建LLM 
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
#定义agent的prompt
#https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")
#定义工具,加载预制的工具,注意有的工具需要提供LLM
tools = load_tools(["llm-math"], llm=llm)
#创建agent
agent = create_openai_functions_agent(llm, tools, prompt)
#定义agent的执行器，这里注意与老版本的不同
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "你好"})

# Agent
- 中间步骤处理
- 提示词
- 模型配置(停止符必要的话)
- 输出解析器
<hr>

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, tool
from langchain.agents.output_parsers import XMLAgentOutputParser
from langchain_openai import ChatOpenAI

In [ ]:
#配置模型
model = ChatOpenAI(
    model="gpt-4-1106-preview",
    temperature=0,
    )
#可用工具
@tool
def search(query: str) -> str:
    """当需要了解最新的天气信息的时候才会使用这个工具。"""
    return "晴朗,32摄氏度,无风"

tool_list = [search]
tool_list

In [ ]:
#提示词模版
# https://smith.langchain.com/hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/xml-agent-convo")
prompt

中间步骤，实现一个log

In [ ]:
def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log

In [ ]:
#将工具列表插入到模版中
def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

定义agent

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | model.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgentOutputParser()
)
agent

In [ ]:
#执行agent
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [ ]:
agent_executor.invoke({"input": "北京今天的天气如何?"})

## agentType更新版
![agtype.png](agtype.png)

一个比较复杂的agent
- 工具
- 检索增强RAG
- 记忆


In [51]:
#定义工具
import os

os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_community.utilities import SerpAPIWrapper

In [ ]:
@tool
def search(query: str) -> str:
    """当需要查找实时信息的时候才会使用这个工具."""
    serp = SerpAPIWrapper()
    return serp.run(query)

print(search.name)
print(search.description)
print(search.args)

In [ ]:
search("北京今天的天气如何?")

In [ ]:
#RAG增强生成
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
print(f"Loaded {len(docs)} documents")
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [ ]:
#搜索匹配文档块
retriever.get_relevant_documents("如何debug?")[0]

In [ ]:
#把检索器加入到工具中
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "搜索有关 LangSmith 的信息。关于LangSmith的任何问题，你一定要使用这个工具！",
)
retriever_tool

In [ ]:
#可用工具集
tools = [search, retriever_tool]
tools

In [ ]:
#定义模型
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [ ]:
#从hub中获取模版
from langchain import hub

# 一个最简单的模版,带记忆
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
#创建agent
from langchain.agents import create_openai_functions_agent
agent = create_openai_functions_agent(llm, tools, prompt)

In [ ]:
#创建agent执行器AgentExecutor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
#执行
agent_executor.invoke({"input": "hi!"})

In [ ]:
agent_executor.invoke({"input": "langsmith如何帮助做项目测试?"})

In [ ]:
#实时信息提问
agent_executor.invoke({"input": "北京今天的天气如何?"})

In [ ]:
#前面的交互都不带记忆
agent_executor.invoke({"input": "截止目前我们都聊了什么?"})

In [ ]:
#交互时添加记忆
agent_executor.invoke({"input": "hi! 我叫bob", "chat_history": []})

In [ ]:
#手动构造记忆数据
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! 我叫bob"),
            AIMessage(content="你好，Bob！很高兴认识你。有什么我可以帮助你的吗？"),
        ],
        "input": "我叫什么名字?",
    }
)

In [ ]:
#使用RunnableWithMessageHistory自动构造记忆数据
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
message_history = ChatMessageHistory()

In [ ]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "hi! 我是tomie"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)

In [ ]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "我叫什么名字?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)

In [ ]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "LangSmith如何使用?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)

In [ ]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "截止目前我们都聊了什么?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)